# **[DONT RUN] Finding Labels (Don't Need To Look At - Commented Out)**

In [1]:
# path = Path("")

# video_parent = path
# split_video_path = path
# split_csv = load_label(path / f'annotations/val.csv')
# split_videos = list(split_video_path.glob('*.mp4'))
# split_videos = {str(p.stem)[:11]:p for p in split_videos}
# split_final = {split_videos[k]:split_csv[k] for k in split_csv.keys() & split_videos.keys()}
# labels = set(split_final.values())
# for label in labels:
#     label_pth = split_video_path / label
#     label_pth.mkdir(exist_ok=True, parents=True)
# split = "val"
# for vid_pth, label in tqdm(split_final.items(), desc=f'Progress {split}'):
#     dst_vid = "/content/drive/MyDrive/COS429/kinetics-dataset/validate" / split_video_path / label / vid_pth.name
#     shutil.move(vid_pth.resolve(), dst_vid)

In [2]:
# from pathlib import Path
# import numpy as np
# path = Path("")
# split_csv = load_label(path / f'annotations/val.csv')
# split_videos = []
# split_videos = list(split_video_path.glob('*.mp4'))
# split_videos = {str(p.stem)[:11]:p for p in split_videos}
# split_final = {split_videos[k]:split_csv[k] for k in split_csv.keys() & split_videos.keys()}
# labels = set(split_final.values())

In [3]:
# def load_label(csv):
#     table = np.loadtxt(csv, skiprows=1, dtype=str, delimiter=',')
#     return {k: v.replace('"', '') for k, v in zip(table[:, 1], table[:, 0])}

In [4]:
# from glob import glob
# labels = glob("*/", recursive = True)
# labels = [sub[: -1] for sub in labels]

In [5]:
# import pickle
# with open("kinetics-dataset/labels.pickle", "rb") as fp:   # Unpickling
#     labels = pickle.load(fp)

In [6]:
# #Be at /content/drive/MyDrive/COS429 when you run this 
# from pathlib import Path
# video_paths = []
# master_video_path = Path("kinetics-dataset/val")
# for label in labels:
#   new_path = master_video_path / label
#   video_paths.extend(list(new_path.glob('*.mp4')))

In [7]:
# with open('kinetics-dataset/video_paths.pickle', 'wb') as f:
#     pickle.dump(video_paths, f)

# **[RUN] Run the Following to Initialize Params for the Baselines**

In [1]:
%load_ext autoreload
%autoreload 2
import pickle
import json

In [2]:
with open('kinetics-dataset/video_paths.pickle', 'rb') as f:
    video_paths = pickle.load(f)

In [3]:
f = open('data/kinetics400/validate/validate.json')
annotations_dict = json.load(f)

In [4]:
from model import VideoMAEModel
from clip_sampler import get_frames_from_video_path
from importlib import reload
import matplotlib.pyplot as plt
import av
import json
from tqdm import trange, tqdm
import numpy as np
from experiments import run_experiment
from experiments_runall import run_experiment_all
np.random.seed(0)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


In [5]:
model = VideoMAEModel()

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


# **Baseline 0.1: Randomly Sample 16 Frames (Sequential)**

In [6]:
# print(video_paths)
run_experiment(model, "random-sequential", video_paths, batch_size = 10)

run experiment


ValueError: unsupported pickle protocol: 5

# **Baseline 0.2: Random Sample 16 Frames (Non-Sequential)**

In [ ]:
run_experiment(model, "completely-random", video_paths)

# **Baseline 1: Randomly Sample 1 Frame of the Video**

In [ ]:
run_experiment(model, "one-random", video_paths)

# **Baseline 2: Get 16 Frames - One Frame From Each Fourth of the Video**


**Basically, we sample one frame from the first 1/4th of the video, one from 1/4th to 1/2th way in the video, one from 1/2 to 3/4th of the video, and the last fourth. We multiply each of those frames by four to get 16 frames.**


In [ ]:
run_experiment(model, "fourths", video_paths)

# **VIDEO VISUALIZATION (ASIDE)**

In [ ]:
import random
random.seed(10)
indices = random.sample(range(0, len(video_paths)), 100)
print(indices)

In [ ]:
video_path = video_paths[400]
video_frames = get_frames_from_video_path("fourths", video_path, 16, 1)
predicted_label, label_num, logits = model.get_predicted_label(video_frames)
actual_label = video_path.parent.name
print(predicted_label)
print(actual_label)

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open(str(video_paths[400]),'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)


# **Baseline 3: Use Optical Flow - Smart Sampling Strategy**

In [ ]:
frames_dict = dict()

In [ ]:
import random
random.seed(10)
indices = random.sample(range(0, len(video_paths)), 100)
print(indices)

In [ ]:
# for index in tqdm(indices):
  

In [ ]:
video_path = video_paths[450]
container = av.open(str(video_path.resolve()))
seg_len = container.streams.video[0].frames

In [ ]:
frames = get_frames_from_video_path("all", video_paths[450], 16, 1).shape

In [ ]:
num_examples = 100
indices = np.arange(start = 0, stop = seg_len, step = int(seg_len/num_examples))
print(indices)
print(len(indices))

In [ ]:
from utils import get_frames_from_container
frames = get_frames_from_container(container, indices, format_type = "gray")

In [ ]:
import cv2 as cv
import time 

start = time.time()
flow_values = []
for i in range(0, len(frames) - 1):
    flow = np.linalg.norm(cv.calcOpticalFlowFarneback(frames[i], frames[i + 1], None, 0.5, 3, 15, 3, 5, 1.2, 0))
    flow_values.append(flow)

end = time.time()
print(end - start)


In [ ]:
import random
plt.plot(flow_values)

# Baseline 4: Object Detection

In [32]:
run_experiment_all(model, "obj-detection-all", video_paths, seed = 10, num_frames=16, num_examples = 250, outer_batch_size = 250, batch_size = 25)

run experiment
Iteration 0:


Evaluating:: 100%|██████████| 10/10 [00:06<00:00,  1.51it/s]


Accuracy for obj-detection-top16: 0.788
Accuracy for obj-detection-low16: 0.728
Accuracy for obj-detection-top8: 0.704
Accuracy for obj-detection-top4: 0.644
Accuracy for obj-detection-top1: 0.576
Accuracy for obj-detection-mixed: 0.8


# Positional Frames

In [6]:
run_experiment_all(model, "position-all", video_paths, seed = 10, num_frames=16, num_examples = 250, outer_batch_size = 250, batch_size = 25)

run experiment
Iteration 0:


Preprocessing::   0%|          | 0/250 [00:00<?, ?it/s]/home/blou/.conda/envs/cos429/lib/python3.7/site-packages/transformers/feature_extraction_utils.py:147: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  return torch.tensor(value)
Evaluating:: 100%|██████████| 10/10 [00:07<00:00,  1.41it/s]


Accuracy for position-fourths: 0.66
Accuracy for position_beginning: 0.748
Accuracy for position_middle: 0.784
Accuracy for position_end: 0.764
Accuracy for position-mixed: 0.816


- Audio
- Pose Detection 
- Non Ordered, Non Sequential

In [ ]:
import pandas as pd
dataset = pd.read_csv("outputs/random-sequential_10.csv")

In [ ]:
dataset

,Actual Label,Predicted Label,Video Path,Video ID,Sampling Strategy,Video Index,Correct
0,sword fighting,sword fighting,kinetics-dataset/val/sword fighting/WGztQjJ0aM...,WGztQjJ0aMU,random-sequential,4680,True
1,grooming horse,grooming horse,kinetics-dataset/val/grooming horse/1RYKyKAYZO...,1RYKyKAYZOQ,random-sequential,266,True
2,crying,crying,kinetics-dataset/val/crying/IfFZalicJ_U_000001...,IfFZalicJ_U,random-sequential,3513,True
3,spraying,spraying,kinetics-dataset/val/spraying/NPYa_aJXLyk_0004...,NPYa_aJXLyk,random-sequential,3953,True
4,playing ice hockey,playing ice hockey,kinetics-dataset/val/playing ice hockey/7D4VHp...,7D4VHpfoPak,random-sequential,4735,True
...,...,...,...,...,...,...,...
95,using remote controller (not gaming),using remote controller (not gaming),kinetics-dataset/val/using remote controller (...,iKJ_R-CB1oI,random-sequential,6333,True
96,digging,blasting sand,kinetics-dataset/val/digging/hBMLJRNjjso_00000...,hBMLJRNjjso,random-sequential,791,False
97,skateboarding,skateboarding,kinetics-dataset/val/skateboarding/hYK_Jcr-N-o...,hYK_Jcr-N-o,random-sequential,3617,True
98,auctioning,making tea,kinetics-dataset/val/auctioning/JzAYnk-gf4Y_00...,JzAYnk-gf4Y,random-sequential,1362,False
